In [19]:
%load_ext legate.jupyter
%legate_info

The legate.jupyter extension is already loaded. To reload it, use:
  %reload_ext legate.jupyter
Kernel 'legate_1gpu' configured for 1 node(s)

Cores:
  CPUs to use per rank : 4
  GPUs to use per rank : 1
  OpenMP groups to use per rank : 0
  Threads per OpenMP group : 4
  Utility processors per rank : 2

Memory:
  DRAM memory per rank (in MBs) : 4000
  DRAM memory per NUMA domain per rank (in MBs) : 0
  Framebuffer memory per GPU (in MBs) : 4000
  Zero-copy memory per rank (in MBs) : 32
  Registered CPU-side pinned memory per rank (in MBs) : 0



In [20]:
import legateboost as lb
from sklearn.datasets import fetch_california_housing
import numpy as np


rs = np.random.RandomState(0)
X, y = fetch_california_housing(return_X_y=True, as_frame=False)

feature_names = ["MedInc", "HouseAge", "AveRooms", "AveBedrms", "Population", "AveOccup",
                 "Latitude", "Longitude"]
feature = 6
X = X[:,feature:feature+1]
n_estimators = 10
n_frames = 40
model = lb.LBRegressor(verbose=True, init='average', max_depth=2, n_estimators=n_estimators, learning_rate=0.1, random_state=rs, objective='normal')
preds = [model.partial_fit(X, y).predict(X) for _ in range(n_frames)]


<__array_function__ internals>:180: RuntimeWarning: cuNumeric has not implemented result_type and is falling back to canonical NumPy. You may notice significantly decreased performance for this function call.
i: 0 train normal_neg_ll: 1.577505904446265
i: 1 train normal_neg_ll: 1.565520985706062
i: 2 train normal_neg_ll: 1.562331272172433
i: 3 train normal_neg_ll: 1.5569464505032224
i: 4 train normal_neg_ll: 1.5544855279913048
i: 5 train normal_neg_ll: 1.547013450535991
i: 6 train normal_neg_ll: 1.5437562094962582
i: 7 train normal_neg_ll: 1.5423819197064748
i: 8 train normal_neg_ll: 1.541499918305991
i: 9 train normal_neg_ll: 1.5407504442426443
i: 10 train normal_neg_ll: 1.540060635798313
i: 11 train normal_neg_ll: 1.5386167910444102
i: 12 train normal_neg_ll: 1.537428103919441
i: 13 train normal_neg_ll: 1.530831623847286
i: 14 train normal_neg_ll: 1.5304336810316708
i: 15 train normal_neg_ll: 1.5279111839072683
i: 16 train normal_neg_ll: 1.5277059585821908
i: 17 train normal_neg_ll: 

In [21]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import animation
from scipy.stats import norm
sns.set()

sample = rs.choice(X.shape[0], 1000, replace=False)
X_test = X[sample]
y_test = y[sample]
feature_name = feature_names[feature]
fig, ax = plt.subplots()

def animate(i):
    ax = fig.axes[0]
    ax.cla()
    ax.set_title("Distribution of House Values: Boosting iterations {}".format((i+1)*n_estimators))
    data = pd.DataFrame( {feature_name: X_test[:, 0],"y":y_test, "Predicted house value": preds[i][sample, 0], "SD": preds[i][sample, 1]}).sort_values(by=feature_name)
    ax = sns.lineplot(x=feature_name, y="Predicted house value", data=data, ax=ax)
    interval = norm.interval(0.95, loc=data["Predicted house value"], scale=data["SD"])
    ax.fill_between(data[feature_name], interval[0], interval[1], alpha=0.2)

    sns.scatterplot(x=feature_name, y="y", data=data, ax=ax, s=15, color=".2", alpha=0.2)
    plt.tight_layout()
    return (fig,)

anim = animation.FuncAnimation(fig, animate, frames=len(preds), interval=500, blit=True)
plt.close()
from IPython.display import clear_output
clear_output()

In [28]:

from IPython.display import HTML
from matplotlib import rcParams
rcParams["animation.codec"] = 'h264'
HTML(anim.to_html5_video())